# Libraries

In [13]:
# General
import pandas as pd
import numpy as np
from math import sqrt

# Clustering algorithm : DBSCAN 
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale

# plot result
import matplotlib.pyplot as plt

# Import Data

In [29]:
# Import Data
credit = pd.read_csv('Data/creditCleanAllBills.csv')

In [24]:
# iterating the columns
colnames = []
for col in credit.columns: 
    colnames.append(col)
# colnames

In [53]:
# Select Features
# see if there are regularities in limit, age and recent bill and payement info
# across other demografic characteristics
creditLimit = credit.loc[: , ['Limit', 'Sex', 'Marriage',
                         'Default']]

creditBill = credit.loc[: , ['Sex', 'Marriage',
                         'BillSep', 'Default']]

creditAge = credit.loc[: , [ 'Sex', 'Marriage', 'Age',
                         'Default']]

creditPaid = credit.loc[: , [ 'Sex', 'Marriage', 
                         'PaidSep',  'Default']]

# Prepare Cluster Alg

In [31]:
# use min/max normalization
# will create 8 groups based on
# sex-marriage-default
# and calculate split off values for the other variables

# MinMax (0-1) normalization
creditMM = minmax_scale(creditSelect)

# turn outcome into df
columns = creditSelect.columns # save names of columns
creditMM = pd.DataFrame(data = creditMM, columns = columns)

In [5]:
# alternative method : scale and centre
# Note : scale is based on mean and standard dev 
# Because scales between variables differs, its hard to make clusters with 
# (DBSCAN's eps value applies to all variables equally)
# more info : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler

## Compute the mean and std
# scaler = StandardScaler().fit(credit)

## transform data with scale
# credit2 = scaler.transform(credit) 

## turn result into df
# columns = credit.columns 
# credit2 = pd.DataFrame(data = credit2, columns = columns)

In [32]:
creditMM.describe()

,Limit,Sex,Marriage,Age,BillSep,BillAug,BillJul,PaidSep,PaidAug,PaidJul,Default
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.159075,0.603733,0.544700,0.249750,0.191846,0.112893,0.112157,0.006483,0.003516,0.005832,0.221200
std,0.131058,0.489129,0.498006,0.158929,0.065159,0.067546,0.038076,0.018961,0.013680,0.019650,0.415062
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.040404,0.000000,0.000000,0.120690,0.149668,0.069053,0.087808,0.001145,0.000495,0.000435,0.000000
50%,0.131313,1.000000,1.000000,0.224138,0.166324,0.086340,0.097374,0.002404,0.001193,0.002009,0.000000
75%,0.232323,1.000000,1.000000,0.344828,0.205887,0.126964,0.119378,0.005731,0.002969,0.005028,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
# Compute DBSCAN

# note: unclear how to get clusters from this .fit() command
# use command below : fit_predict()

db = DBSCAN(eps = 0.4, min_samples = 100).fit(creditMM)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)


print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)



Estimated number of clusters: 8
Estimated number of noise points: 8


In [34]:
# save cluster info in a variable
creditMM["cluster"] = db.fit_predict(creditMM)

In [35]:
credit['cluster'] = creditMM['cluster']

In [41]:
# check distribution : obs / cluster
credit.groupby(["cluster"]).count()

,Limit,Sex,Education,Marriage,Age,StatusSep,StatusAug,StatusJul,StatusJun,StatusMay,...,BillJun,BillMay,BillApr,PaidSep,PaidAug,PaidJul,PaidJun,PaidMay,PaidApr,Default
cluster,,,,,,,,,,,,,,,,,,,,,
-1,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
0,1860,1860,1860,1860,1860,1860,1860,1860,1860,1860,...,1860,1860,1860,1860,1860,1860,1860,1860,1860,1860
1,1903,1903,1903,1903,1903,1903,1903,1903,1903,1903,...,1903,1903,1903,1903,1903,1903,1903,1903,1903,1903
2,7738,7738,7738,7738,7738,7738,7738,7738,7738,7738,...,7738,7738,7738,7738,7738,7738,7738,7738,7738,7738
3,6606,6606,6606,6606,6606,6606,6606,6606,6606,6606,...,6606,6606,6606,6606,6606,6606,6606,6606,6606,6606
4,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,...,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843
5,5170,5170,5170,5170,5170,5170,5170,5170,5170,5170,...,5170,5170,5170,5170,5170,5170,5170,5170,5170,5170
6,1527,1527,1527,1527,1527,1527,1527,1527,1527,1527,...,1527,1527,1527,1527,1527,1527,1527,1527,1527,1527
7,1345,1345,1345,1345,1345,1345,1345,1345,1345,1345,...,1345,1345,1345,1345,1345,1345,1345,1345,1345,1345


In [42]:
# check mean values in clusters
creditMM.groupby(['cluster']).mean()

# so what did the cluster do?
# it checks common payment/billing/status/age values for :
# groups that can be created through binary values
# married men that default
# married women that default
# married women that pay
# etc.
# and splits the population based on those values
# ! importantly ! The values do not represent an approximation of the real values of
# observations in a group. They represent splitting criteria.

,Limit,Sex,Marriage,Age,BillSep,BillAug,BillJul,PaidSep,PaidAug,PaidJul,Default
cluster,,,,,,,,,,,
-1,0.511364,0.625,0.375,0.323276,0.446554,0.429976,0.417701,0.257312,0.391169,0.454046,0.125
0,0.129277,1.000,0.000,0.310187,0.188958,0.110523,0.111068,0.003861,0.002110,0.003604,1.000
1,0.119975,1.000,1.000,0.170792,0.186959,0.108559,0.110086,0.003863,0.001913,0.003497,1.000
2,0.164508,1.000,1.000,0.177349,0.189797,0.110760,0.111104,0.006935,0.003589,0.006091,0.000
3,0.179328,1.000,0.000,0.308176,0.191468,0.112431,0.111933,0.007196,0.003780,0.006107,0.000
4,0.197159,0.000,0.000,0.358058,0.198678,0.118930,0.115701,0.007799,0.004392,0.007004,0.000
5,0.144734,0.000,1.000,0.197519,0.193084,0.114085,0.112216,0.006866,0.003746,0.006298,0.000
6,0.095227,0.000,1.000,0.193099,0.187786,0.109727,0.109931,0.003620,0.001907,0.003810,1.000
7,0.141579,0.000,0.000,0.362646,0.195211,0.116721,0.114109,0.004250,0.002135,0.003863,1.000


In [43]:
dummies = pd.get_dummies(creditMM["cluster"])

In [48]:
# Save clusters

# as dummies
dummies.to_csv('Data/clusterDummies.csv', index = False)

# and as a single variable
creditMM["cluster"].to_csv('Data/clusterVariable.csv', index = False)

C:\Users\sbvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  import sys


In [46]:
# concatenate the original credit df with the dummies df
creditCluster = pd.concat([credit, dummies.loc[:, '0': '7']], axis = 1, sort = False)

In [51]:
# heat map : correlation matrix
# note : cluster are not highly correlated with status

creditCluster.corr().style.background_gradient(cmap = 'seismic',
                                               axis = None,
                                               low = 0.16).set_precision(2)

,Limit,Sex,Education,Marriage,Age,StatusSep,StatusAug,StatusJul,StatusJun,StatusMay,StatusApr,BillSep,BillAug,BillJul,BillJun,BillMay,BillApr,PaidSep,PaidAug,PaidJul,PaidJun,PaidMay,PaidApr,Default,cluster,0,1,2,3,4,5,6,7
Limit,1,0.025,-0.23,-0.1,0.14,-0.27,-0.3,-0.29,-0.27,-0.25,-0.24,0.29,0.28,0.28,0.29,0.3,0.29,0.2,0.18,0.21,0.2,0.22,0.22,-0.15,-0.019,-0.058,-0.078,0.024,0.082,0.11,-0.05,-0.11,-0.029
Sex,0.025,1,0.014,-0.03,-0.091,-0.058,-0.071,-0.066,-0.06,-0.055,-0.044,-0.034,-0.031,-0.025,-0.022,-0.017,-0.017,-0.00024,-0.0014,-0.0086,-0.0022,-0.0017,-0.0028,-0.04,-0.84,0.21,0.21,0.48,0.43,-0.47,-0.56,-0.29,-0.27
Education,-0.23,0.014,1,-0.16,0.18,0.11,0.13,0.12,0.12,0.1,0.089,0.017,0.012,0.0067,-0.0061,-0.012,-0.013,-0.041,-0.033,-0.044,-0.041,-0.045,-0.044,0.034,0.003,0.07,-0.034,-0.11,0.11,0.026,-0.055,-0.0051,0.032
Marriage,-0.1,-0.03,-0.16,1,-0.45,0.016,0.023,0.031,0.03,0.034,0.032,-0.025,-0.022,-0.026,-0.023,-0.025,-0.021,-0.007,-0.011,-0.0053,-0.015,-0.0022,-0.006,-0.03,-0.017,-0.28,0.24,0.54,-0.58,-0.42,0.42,0.21,-0.24
Age,0.14,-0.091,0.18,-0.45,1,-0.039,-0.05,-0.053,-0.05,-0.054,-0.049,0.056,0.054,0.054,0.051,0.049,0.048,0.026,0.022,0.029,0.021,0.023,0.019,0.014,0.09,0.098,-0.13,-0.27,0.2,0.26,-0.15,-0.083,0.15
StatusSep,-0.27,-0.058,0.11,0.016,-0.039,1,0.67,0.57,0.54,0.51,0.47,0.19,0.19,0.18,0.18,0.18,0.18,-0.079,-0.07,-0.071,-0.064,-0.058,-0.059,0.32,0.028,0.15,0.15,-0.12,-0.12,-0.073,-0.029,0.16,0.14
StatusAug,-0.3,-0.071,0.13,0.023,-0.05,0.67,1,0.77,0.66,0.62,0.58,0.23,0.24,0.22,0.22,0.22,0.22,-0.081,-0.059,-0.056,-0.047,-0.037,-0.037,0.26,0.041,0.12,0.11,-0.099,-0.11,-0.051,-0.0084,0.14,0.11
StatusJul,-0.29,-0.066,0.12,0.031,-0.053,0.57,0.77,1,0.78,0.69,0.63,0.21,0.24,0.23,0.23,0.23,0.22,0.0013,-0.067,-0.053,-0.046,-0.036,-0.036,0.24,0.044,0.094,0.095,-0.083,-0.1,-0.053,-0.0045,0.14,0.1
StatusJun,-0.27,-0.06,0.12,0.03,-0.05,0.54,0.66,0.78,1,0.82,0.72,0.2,0.23,0.24,0.25,0.24,0.24,-0.0094,-0.0019,-0.069,-0.043,-0.034,-0.027,0.22,0.041,0.087,0.091,-0.076,-0.095,-0.052,6.4e-05,0.12,0.1
StatusMay,-0.25,-0.055,0.1,0.034,-0.054,0.51,0.62,0.69,0.82,1,0.82,0.21,0.23,0.24,0.27,0.27,0.26,-0.0061,-0.0032,0.0091,-0.058,-0.033,-0.023,0.2,0.038,0.078,0.086,-0.065,-0.093,-0.05,-0.0032,0.12,0.094
